In [1]:
import pandas as pd
import datetime

pd.options.mode.chained_assignment = None  # default='warn'

do_head = ('FW_TS','FW_LD','FW_FS','FW_TB','SYS_FS','SYS_EA','SYS_CHK','RT_RST')
#这些循环作业从0点开始

In [2]:
d_list= ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
          '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
          '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
          '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
          '180711', '180712', '180713', '180714', '180715']


In [ ]:

condition_o = pd.read_csv(r'E:\jupyter\CTM\data_src\condition_o\conditin_o_%s.csv'%day)

if day<'181001':
    condition_o['odate'] = condition_o['odate'].apply(lambda x:'0'+str(x))
else:
    condition_o['odate'] = condition_o['odate'].apply(str)
    
day_i = day[-4:]

#去重
print(condition_o.shape)
condition_o = condition_o[condition_o.odate==day_i]
print(condition_o.shape)
condition_o.drop_duplicates(subset=['up','down'],keep='last')
print(condition_o.shape)



In [4]:
day = '180605'

day_begain = int('20'+day+'000000')
day_4_hour = int('20'+day+'040000')

dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')

def f_get_end_dt(x):
    dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
    dt = (dat_t-dat_start).total_seconds()//60
    return dt

def cnv_start_time(x):
    return x//100*60+x%100




In [ ]:
day = '180605'

day_begain = int('20'+day+'000000')
day_4_hour = int('20'+day+'040000')

dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')


def f_get_mint_bef(x):
    dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
    dt = (dat_t-dat_start).total_seconds()//60
    return dt

condition_i = pd.read_csv(r'E:\jupyter\CTM\data_src\condition_i\conditin_i_%s.csv'%day)
condition_i['odate'] = condition_i['odate'].apply(lambda x:'0'+str(x))
day_i = day[-4:]
condition_i = condition_i[condition_i.odate ==day_i]
condition_i.drop_duplicates(subset=['up','down'],keep='fisrt')

evet_i = pd.read_csv(r'E:\jupyter\CTM\data_processed\evet\evet_dt_%s.csv'%day)

ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%day,low_memory=False)
ajob = ajob[ajob['order_time']<=day_4_hour]
ajob = ajob[ajob.odate==day_int]
ajob.drop_duplicates('job_name', keep='last', inplace=True)


In [15]:
day = '180607'

day_begain = int('20'+day+'000000')
dat_start = datetime.datetime.strptime(str(day_begain),'%Y%m%d%H%M%S')


def f_get_mint_bef(x):
    dat_t = datetime.datetime.strptime(str(x),'%Y%m%d%H%M%S')
    dt = (dat_t-dat_start).total_seconds()//60
    return dt

condition_i = pd.read_csv(r'E:\jupyter\CTM\data_src\condition_i\conditin_i_%s.csv'%day)
condition_i['odate'] = condition_i['odate'].apply(lambda x:'0'+str(x))

day_i = day[-4:]
#去重
print(condition_i.shape)
condition_i = condition_i[condition_i.odate==day_i]
print(condition_i.shape)
condition_i.drop_duplicates(subset=['up','down'],keep='last')
print(condition_i.shape)

(266403, 7)
(249649, 7)
(249649, 7)


In [16]:
condition = condition_i[['up','down']]

In [17]:
day_4_hour = int('20'+day+'040000')
day_int = int(day)
ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob\ajob_%s_0.txt'%day,low_memory=False)
ajob = ajob[ajob['order_time']<=day_4_hour]
ajob = ajob[ajob.odate==day_int]
ajob = ajob[ajob.task_type!= 'SMART Table']
ajob.drop_duplicates('job_name', keep='last', inplace=True)
print(ajob.shape)

(242855, 32)


In [26]:
evet_i = pd.read_csv(r'E:\jupyter\CTM\data_processed\evet\evet_dt_%s.csv'%day)

In [27]:

ajob = ajob[ajob.end_time.notnull()]
print(ajob.shape)
ajob['end_time'] = ajob['end_time'].apply(int)
ajob['end_dt'] = ajob['end_time'].apply(f_get_end_dt)

#全量表
ajob_ = ajob[['job_name','end_dt']]
ajob_.rename(columns={"job_name":"up"}, inplace = True)

#有开始时间表
ajob_f_t = ajob[ajob.from_time>1][['job_name','from_time']]
ajob_f_t['from_time'] = ajob_f_t['from_time'].apply(cnv_start_time)
ajob_f_t['up'] = '--'

ajob_f_t.rename(columns={"job_name":"down"}, inplace = True)
ajob_f_t.rename(columns={"from_time":"end_dt"}, inplace = True)


condition_m = condition.merge(ajob_, on='up',how='left')
condition_m = pd.concat([condition_m,ajob_f_t], ignore_index=True)

#取上游作业的end_dt,最大值
condition_l = condition_m.groupby(['down']).end_dt.max()
condition_l = condition_l.to_frame(name='up_end_dt_max')
condition_l['job_name'] = condition_l.index



evet_i = evet_i[evet_i['end_time'].notnull()]
evet_i['end_time'] = evet_i['end_time'].apply(int)
evet_i['end_dt'] = evet_i['end_time'].apply(f_get_end_dt)

#过滤出folder 不是'SMART Table'
evet_i_m = evet_i_m[(evet_i_m.end_time<20180607000000)&(evet_i_m.end_time>20180605000000)]
evet_i = evet_i.merge(ajob[['job_name','task_type']],on='job_name',how='left')
evet_i = evet_i[evet_i['task_type']!='SMART Table']
evet_i = evet_i[evet_i['task_type'].notnull()]

evet_i_m = evet_i.merge(condition_l,on='job_name',how='left')

#evet_i_m['end_dt_max'] = evet_i_m['end_dt_max'].fillna(0)
#evet_i_m['end_dt_sum'] = evet_i_m[['end_dt_max','dt']].apply(lambda x:x[0]+x[1],axis=1)
evet_i_m.to_csv('make_sure_dt.csv',index=False)


(241501, 33)


In [28]:
evet_i_m[evet_i_m.end_time>20180605040000].to_csv('make_sure_dt_4.csv',index=False)


In [20]:
evet_i.head()

,max_t,min_t,dt,order_id,job_name,start_time,end_time,task_type_x,end_dt,task_type_y
0,2018-06-05 15:21:12,2018-06-05 15:21:12,0.000,138ho3,LV05_CMD_PRO,2.018061e+13,20180605152111,NaN,921.0,NaN
1,2018-06-05 01:26:14,2018-06-05 01:26:12,0.033,138hp3,CMD_DB_ETL_PRO_TABRTNSQL_RTN_ACMD_DIM_SPP_REL,2.018061e+13,20180605012614,Job,86.0,Job
2,2018-06-05 01:26:13,2018-06-05 01:26:11,0.033,138hq3,CMD_DB_ETL_PRO_TABRTNSQL_RTN_ACMD_DIM_SPP_REL_HIS,2.018061e+13,20180605012613,Job,86.0,Job
3,2018-06-05 01:26:17,2018-06-05 01:26:14,0.050,138hr3,CMD_DB_ETL_PRO_TABRTNSQL_RTN_ACMD_DIM_SPP_RES,2.018061e+13,20180605012616,Job,86.0,Job
4,2018-06-05 01:31:17,2018-06-05 01:31:15,0.033,138hs3,CMD_DB_ETL_PRO_TABRTNSQL_RTN_ACMD_DIM_SPP_RES_HIS,2.018061e+13,20180605013117,Job,91.0,Job


In [11]:
ajob.head()

,order_id,task_type,job_name,job_id,odate,max_wait,confirm_flag,from_time,to_time,interval,...,nodegroup,order_table,order_time,avg_start_time,cpu_time,prev_odate_rerun_counter,v_date,order_d,server,end_dt
0,4bdxe,Command,EXM_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_D,198188.0,180605,1,0,NaN,NaN,00005M,...,GRP_DS_AGENT,DXP_EX_RTL2_LF06,20180605000155,12518.0,41,NaN,180605,180605,1,98.0
1,4be82,Command,SYS_FW_LD_TDW_1_P_HWHC_3_WEAPP2_WE2_CUS_ADD_OT...,27178.0,180605,1,0,NaN,NaN,NaN,...,GRP_FS_AGENT,DXP_LD_TDW_RTL2_LF06,20180605000155,13018.0,0,NaN,180605,180605,1,114.0
2,4bdxd,Command,EXM_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_A,13977.0,180605,1,0,NaN,NaN,00005M,...,GRP_DS_AGENT,DXP_EX_RTL2_LF06,20180605000155,12545.0,18,NaN,180605,180605,1,98.0
3,4beg2,Command,EXZ_T_HTDW_AMDS_AMDS_FTM_MN010202,47648.0,180605,1,0,NaN,NaN,00002M,...,GRP_DS_AGENT,DXP_EX_CMD_LV05,20180605000155,104052.0,28,NaN,180605,180605,1,717.0
4,4be02,Command,EXM_P_HWHC_5_WEAPP4_WE2_CUS_ADD_OTHER_AST_D,41420.0,180605,1,0,NaN,NaN,00005M,...,GRP_DS_AGENT,DXP_EX_RTL2_LF06,20180605000155,12618.0,21,NaN,180605,180605,1,108.0


In [12]:
print(ajob_f_t.head())

                                   job_name  from_time
107              EXZ_P_HBDS_1_SYSCAT_TABLES      600.0
108     EXM_P_HFAP_1_SYSTEM_ALL_TAB_COLUMNS      600.0
111  EXM_P_HWPH_3_INFORMATION_SCHEMA_TABLES        6.0
115              EXZ_P_HCFC_1_SYSCAT_TABLES      600.0
118  EXM_P_HWPD_5_INFORMATION_SCHEMA_TABLES        6.0


In [17]:
ajob_f_t.head()

,down,end_dt,up
107,EXZ_P_HBDS_1_SYSCAT_TABLES,360.0,--
108,EXM_P_HFAP_1_SYSTEM_ALL_TAB_COLUMNS,360.0,--
111,EXM_P_HWPH_3_INFORMATION_SCHEMA_TABLES,6.0,--
115,EXZ_P_HCFC_1_SYSCAT_TABLES,360.0,--
118,EXM_P_HWPD_5_INFORMATION_SCHEMA_TABLES,6.0,--


In [30]:
condition_m[condition_m['end_dt'].notnull()].shape

(241406, 3)

In [31]:
condition_m.head()

,up,down,end_dt
0,EX_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_D,EXM_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_D,96.0
1,LD_TDW_1_P_HWHC_3_WEAPP2_WE2_CUS_ADD_OTHER_AST_A,SYS_FW_LD_TDW_1_P_HWHC_3_WEAPP2_WE2_CUS_ADD_OT...,114.0
2,EX_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_A,EXM_P_HWHC_5_WEAPP4_WE2_CUS_EXAM_REFER_INFO_A,96.0
3,EXM_T_HTDW_AMDS_AMDS_FTM_MN010202,EXZ_T_HTDW_AMDS_AMDS_FTM_MN010202,717.0
4,FW_LD_CMD_1_T_HTDW_AMDS_AMDS_FTM_MN010102,LD_CMD_1_T_HTDW_AMDS_AMDS_FTM_MN010102,685.0


In [34]:
condition_m.shape

(284660, 3)

In [36]:
condition_l.head()

down
78LET_ZXTJ_RTL2                360.0
AAKP_C_AAKP_A_BUSMID_S_RPT    1016.0
AAKP_C_AAKP_M_BILSEL_STS_S     794.0
AAKP_C_AAKP_M_BIL_INCOME_S     240.0
AAKP_C_AAKP_M_BIL_STS_S        249.0
Name: end_dt, dtype: float64

In [39]:
condition_l.head()

,end_dt_max,job_name
down,,
78LET_ZXTJ_RTL2,360.0,78LET_ZXTJ_RTL2
AAKP_C_AAKP_A_BUSMID_S_RPT,1016.0,AAKP_C_AAKP_A_BUSMID_S_RPT
AAKP_C_AAKP_M_BILSEL_STS_S,794.0,AAKP_C_AAKP_M_BILSEL_STS_S
AAKP_C_AAKP_M_BIL_INCOME_S,240.0,AAKP_C_AAKP_M_BIL_INCOME_S
AAKP_C_AAKP_M_BIL_STS_S,249.0,AAKP_C_AAKP_M_BIL_STS_S


In [41]:
condition_l.shape

(192686, 2)

In [44]:
print(evet_i.shape)

print(evet_i.shape)

(247689, 7)
(247642, 8)


In [50]:
print(evet_i_m.shape)
evet_i_m = evet_i_m[evet_i_m.end_time<20180607000000]
print(evet_i_m.shape)

(247642, 10)
(247391, 10)


In [2]:
s='AAKP_C_AAKP_A_BUSMID_S_RPt'
'_'.join(s.split('_')[:2])

'AAKP_C'

In [ ]:
FW_TS  --
FW_LD  -- 无开始作业 开始从0点
FW_FS  -- 无开始作业 开始从0点
FW_TB  ---

SYS_FS  -- 无开始作业 开始从0点
SYS_EA  --
SYS_CHK  --

RT_RST  -- 无开始作业 开始从0点  有时间窗口

这些作业没有上游作业

SYS_FW有上游作业